In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from enadepy.loaders import read_interm
from enadepy.helpers import list_cols_socioecon
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

In this pre-processing step, we use a previously filtered data containing only registries for students from private institutions.

Here, we will select variables of interest by hand, as an initial automated approach did not show very interesting results.

Related notebooks:

    - 3.1-mc-preprocess-pub
    - 4.1-mc-mining-pub

In [3]:
df = read_interm('../data/interim/microdados_enade_ata_2016a2018_priv.csv')

In [4]:
df.head()

,NU_ANO,NT_GER,CO_IES,CO_GRUPO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,QE_I01,QE_I02,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I10,QE_I11,QE_I12,QE_I13,QE_I14,QE_I15,QE_I17,QE_I18,QE_I19,QE_I20,QE_I21,QE_I22,QE_I23,QE_I24,QE_I25
0,2016,44.9,322,19,21,F,2012,2013,A,A,C,D,A,A,A,C,E,E,A,A,A,A,A,A,A,B,B,B,B,E,A
1,2016,57.6,322,19,29,F,2012,2013,A,A,A,A,B,C,A,E,E,E,A,A,A,A,A,A,A,C,B,B,B,E,H
2,2016,65.2,322,19,21,F,2012,2013,A,A,C,B,B,D,A,B,A,C,A,A,A,A,A,A,B,C,A,B,B,E,A
3,2016,65.6,322,19,22,F,2012,2013,A,A,D,D,B,D,C,E,D,C,A,A,A,A,A,A,B,C,B,C,C,E,E
4,2016,54.4,322,19,35,M,2012,2013,A,A,A,A,B,C,B,E,E,E,A,A,A,A,A,A,B,A,B,B,B,E,A


In [5]:
df.shape

(1547, 31)

In [6]:
socio_cols = [col for col in df.columns if col.startswith('QE_')]

See all questions from the socioeconomics questionnaire and their results

In [7]:
dfmelt = df[socio_cols].melt(var_name="Question", value_name="Choice")

In [8]:
pd.crosstab(index=dfmelt['Question'], columns=dfmelt['Choice'])

Choice,A,B,C,D,E,F,G,H,I,J,K
Question,,,,,,,,,,,
QE_I01,1269,224,20,6,28,0,0,0,0,0,0
QE_I02,1112,83,45,298,1,8,0,0,0,0,0
QE_I04,48,307,281,634,231,46,0,0,0,0,0
QE_I05,26,257,306,613,272,73,0,0,0,0,0
QE_I06,84,1150,275,36,0,2,0,0,0,0,0
QE_I07,91,219,414,499,232,62,16,14,0,0,0
QE_I08,212,503,398,188,166,72,8,0,0,0,0
QE_I09,125,461,498,165,208,90,0,0,0,0,0
QE_I10,622,124,107,141,553,0,0,0,0,0,0


In [9]:
socio_sel_cols = ['QE_I02', 'QE_I04', 'QE_I05', 'QE_I06', 'QE_I07', 'QE_I08', 'QE_I09', 'QE_I17', 'QE_I22', 'QE_I23', 'QE_I25']

In [10]:
cols = socio_sel_cols + ['TP_SEXO', 'NT_GER']

In [11]:
cols

['QE_I02',
 'QE_I04',
 'QE_I05',
 'QE_I06',
 'QE_I07',
 'QE_I08',
 'QE_I09',
 'QE_I17',
 'QE_I22',
 'QE_I23',
 'QE_I25',
 'TP_SEXO',
 'NT_GER']

### Split data according to skin color

This split is necessary because there is a dominance of white students in the dataset. By splitting the data, we can look for patterns when students are not  white.

In [12]:
df.QE_I02.value_counts()

A    1112
D     298
B      83
C      45
F       8
E       1
Name: QE_I02, dtype: Int64

In [13]:
df_white = df.query('QE_I02 == "A"')[cols].drop(columns="QE_I02")
df_white

,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I17,QE_I22,QE_I23,QE_I25,TP_SEXO,NT_GER
0,C,D,A,A,A,C,A,B,B,A,F,44.9
1,A,A,B,C,A,E,A,B,B,H,F,57.6
2,C,B,B,D,A,B,A,B,B,A,F,65.2
3,D,D,B,D,C,E,A,C,C,E,F,65.6
4,A,A,B,C,B,E,A,B,B,A,M,54.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1540,E,E,B,B,B,C,A,E,C,A,M,16.4
1542,B,C,A,A,B,C,A,E,D,E,M,48.5
1544,E,D,B,C,D,B,E,B,B,E,F,48.0
1545,D,F,A,A,A,C,B,C,B,H,F,36.2


In [14]:
df_black = df.query('QE_I02 in ["B", "D", "E"]')[cols].drop(columns="QE_I02")
df_black

,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I17,QE_I22,QE_I23,QE_I25,TP_SEXO,NT_GER
13,C,B,C,D,B,A,A,D,E,A,F,45.0
24,D,D,C,D,B,C,A,E,E,A,F,28.4
26,D,D,C,B,C,D,A,E,E,E,F,39.3
39,C,D,B,D,C,B,A,E,D,H,F,43.1
40,D,F,B,D,E,A,A,E,E,D,F,42.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1523,A,B,B,B,B,C,A,B,D,A,F,49.0
1525,C,D,C,D,B,E,A,C,B,E,M,23.9
1526,D,B,C,C,E,F,A,E,B,E,M,59.8
1541,E,D,B,G,E,C,B,B,B,B,F,16.4


## Transforming data

In [15]:
for df_final, sufix in zip([df_white, df_black], ["white", "nowhite"]):
    df_final['NT_GER'] = pd.qcut(df_final.NT_GER, 2, labels=["Q0", "Q1"])
    cats = df_final.columns
    enc = preprocessing.OneHotEncoder(sparse=False)
    # fit and transform in one call and print categories
    out_enc = enc.fit_transform(df_final[cats])
    new_cols = enc.get_feature_names(cats).tolist()
    print(new_cols)
    # create temp. dataframe for concatenation with original data
    df_enc = pd.DataFrame(data=out_enc, columns=new_cols)
    df_enc.index = df_final.index
    # drop original columns and concatenate new encoded columns
    df_final.drop(cats, axis=1, inplace=True)
    df_final = pd.concat([df_final, df_enc], axis=1)
    print(df_final.columns)
    df_final.to_csv(f'../data/preprocessed/enade_2016a2018_priv_onehot_{sufix}.csv', index=False)

['QE_I04_A', 'QE_I04_B', 'QE_I04_C', 'QE_I04_D', 'QE_I04_E', 'QE_I04_F', 'QE_I05_A', 'QE_I05_B', 'QE_I05_C', 'QE_I05_D', 'QE_I05_E', 'QE_I05_F', 'QE_I06_A', 'QE_I06_B', 'QE_I06_C', 'QE_I06_D', 'QE_I07_A', 'QE_I07_B', 'QE_I07_C', 'QE_I07_D', 'QE_I07_E', 'QE_I07_F', 'QE_I07_G', 'QE_I07_H', 'QE_I08_A', 'QE_I08_B', 'QE_I08_C', 'QE_I08_D', 'QE_I08_E', 'QE_I08_F', 'QE_I08_G', 'QE_I09_A', 'QE_I09_B', 'QE_I09_C', 'QE_I09_D', 'QE_I09_E', 'QE_I09_F', 'QE_I17_A', 'QE_I17_B', 'QE_I17_C', 'QE_I17_D', 'QE_I17_E', 'QE_I17_F', 'QE_I22_A', 'QE_I22_B', 'QE_I22_C', 'QE_I22_D', 'QE_I22_E', 'QE_I23_A', 'QE_I23_B', 'QE_I23_C', 'QE_I23_D', 'QE_I23_E', 'QE_I25_A', 'QE_I25_B', 'QE_I25_C', 'QE_I25_D', 'QE_I25_E', 'QE_I25_G', 'QE_I25_H', 'TP_SEXO_F', 'TP_SEXO_M', 'NT_GER_Q0', 'NT_GER_Q1']
Index(['QE_I04_A', 'QE_I04_B', 'QE_I04_C', 'QE_I04_D', 'QE_I04_E', 'QE_I04_F',
       'QE_I05_A', 'QE_I05_B', 'QE_I05_C', 'QE_I05_D', 'QE_I05_E', 'QE_I05_F',
       'QE_I06_A', 'QE_I06_B', 'QE_I06_C', 'QE_I06_D', 'QE_I07_A', 'Q